# Data Cleaning

Import Collected Data from CSV

In [ ]:
import pandas as pd

# read in API gathered and web scraped data from csv
df = pd.read_csv('housing_data.csv')
df.head()

<ipython-input-4-10e5a2d2d5ef>:2: DtypeWarning: Columns (12,39,63,64,80) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('housing_data-2.csv')


,Unnamed: 0,homeData.propertyId,homeData.listingId,homeData.listingDisplayLevel,homeData.mlsId,homeData.url,homeData.dataSourceId,homeData.marketId,homeData.mlsStatusId,homeData.servicePolicyId,...,homeData.bathInfo.rawThreeQuarterBaths,homeData.brokers.listingBrokerAndAgent.brokerName,homeData.directAccessInfo.deviceManufacturer,zip,population,density,added,sold,days_on_market,description
0,1,194790217,122530509,1,502813,/CA/San-Francisco/2437-2441-Post-St-94115/home...,9,2,3563,4,...,NaN,NaN,NaN,94115,32715.0,11115.5,2020-07-31,2020-07-29,-2,For Comp Sale Purposes. .. Prime Lower Pacific...
1,4,1246769,126864052,1,511235,/CA/San-Francisco/195-Dalewood-Way-94127/home/...,9,2,3563,4,...,NaN,NaN,NaN,94127,20710.0,4287.6,2020-12-29,2021-01-20,22,This gorgeous Mid Century Sherwood Forest home...
2,5,1325307,130581142,1,421533081,/CA/San-Francisco/3228-Santiago-St-94116/home/...,9,2,3563,4,...,NaN,NaN,NaN,94116,42523.0,6499.5,2021-04-02,2021-04-23,21,Single family beauty! Newly remodeled three be...
3,6,194684980,199443193,1,81982862,/CA/San-Francisco/6517-California-St-94121/hom...,9,2,3563,4,...,NaN,NaN,NaN,94121,42987.0,6548.4,2025-02-11,2024-11-07,-96,"Nestled in the heart of Sea Cliff, this charmi..."
4,7,1805475,119213680,1,497208,/CA/San-Francisco/318-Ortega-St-94122/home/180...,9,2,3563,4,...,NaN,NaN,NaN,94122,57369.0,6800.4,2020-05-05,2020-04-22,-13,For comp purposes only. Classic tunnel entranc...


Column Renaming and Cleaning

In [ ]:
# rename problematic column names
df = df.rename(columns={'homeData.priceInfo.amount': 'price', 'homeData.lotSize.amount': 'lotSize', 'homeData.sqftInfo.amount': 'sqft'})

In [ ]:
# just get end of column names
new_columns = []
for col in df.columns:
    new_columns.append(col.split('.')[-1])
df.columns = new_columns

In [ ]:
# select important columns, discarding ones that aren't useful
relevant = ['url', 'lastSoldDate', 'listingAddedDate', 'beds', 'price', 'lotSize', 'baths', 'yearBuilt', 'sqft', 'population', 'density', 'description', 'longitude', 'latitude']
df = df[relevant]

Clean and Calculate Dates

In [ ]:
# clean a few dates and calculate how long the house was on market before selling
df['added'] = pd.to_datetime(df['listingAddedDate'].str[:10])
df['sold'] = pd.to_datetime(df['lastSoldDate'].str[:10])
df.dropna(subset=['added', 'sold'], inplace=True)
df['days_on_market'] = (df['sold'] - df['added']).dt.days

In [ ]:
# remove erroneous data
df = df[df['days_on_market'] > 0]

In [ ]:
# make new column based on the date listed
df['yearListed'] = df['listingAddedDate'].str[:4]

In [ ]:
# make new column based on the date listed
df['monthListed'] = df['listingAddedDate'].str[5:7]
df['monthListed'] = df['monthListed'].astype(int)

Show DataFrame

In [ ]:
df

,url,lastSoldDate,listingAddedDate,beds,price,lotSize,baths,yearBuilt,sqft,days_on_market,...,description,description,longitude,latitude,seller_agent,seller_broker,buyer_agent,buyer_broker,yearListed,monthListed
1,/CA/San-Francisco/195-Dalewood-Way-94127/home/...,2021-01-20T08:00:00Z,2020-12-29T20:52:44.508Z,3.0,1575000,4532.0,2.0,1963.0,1503.0,22,...,Pacific Time in the United States,This gorgeous Mid Century Sherwood Forest home...,-122.457494,37.738334,,Redfin,Denis Lancerin,Vanguard Properties,2020,12
2,/CA/San-Francisco/3228-Santiago-St-94116/home/...,2021-04-23T07:00:00Z,2021-04-02T17:34:02.324Z,3.0,1452500,2495.0,1.0,1941.0,1000.0,21,...,Pacific Time in the United States,Single family beauty! Newly remodeled three be...,-122.502106,37.743938,Ali Mafi,Redfin,Kent Chen,Compass,2021,4
7,/CA/San-Francisco/663-5th-Ave-94118/home/8190506,2020-06-10T07:00:00Z,2020-05-15T08:00:21.671Z,2.0,1125000,NaN,1.0,1907.0,968.0,26,...,Pacific Time in the United States,Classic Inner Richmond Edwardian flat! Stunnin...,-122.463173,37.776095,Ali Mafi,Redfin,Miriam Westberg,Redfin,2020,5
8,/CA/San-Francisco/55-Page-St-94102/unit-715/ho...,2020-06-08T07:00:00Z,2020-05-26T07:11:31.395Z,1.0,1117000,NaN,1.0,2008.0,885.0,13,...,Pacific Time in the United States,Views for miles!! This coveted high-floor unit...,-122.421877,37.773890,Gabrielle Bunker,Redfin,Steven Tam,RE/MAX Gold,2020,5
9,/CA/San-Francisco/2127-Balboa-St-94121/home/10...,2022-10-28T07:00:00Z,2022-10-02T05:56:58.367Z,2.0,1300000,2500.0,1.0,1934.0,984.0,26,...,Pacific Time in the United States,On the market for the first time in over 60 ye...,-122.481817,37.776244,Alex Sobieski,Redfin,Dylan Hunter,Compass,2022,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17467,/CA/San-Francisco/1-Daniel-Burnham-Ct-94109/un...,2023-11-17T08:00:00Z,2023-09-21T19:34:43.197Z,1.0,710000,46157.0,1.0,1988.0,836.0,57,...,Pacific Time in the United States,"Introducing a charming one bedroom, one bathro...",-122.421906,37.786783,Emily Tam,Berkshire Hathaway-Franciscan,Samantha Yu,Compass,2023,9
17468,/CA/San-Francisco/260-King-St-94107/unit-815/h...,2021-08-04T07:00:00Z,2021-06-18T02:22:10.698Z,1.0,820000,NaN,1.0,2004.0,903.0,47,...,Pacific Time in the United States,Enjoy luxury living at the Beacon! This stunni...,-122.393672,37.777627,Burton Louie,Compass,Alana George,Intero Real Estate Services,2021,6
17469,/CA/San-Francisco/44-Everglade-Dr-94132/home/1...,2020-07-29T07:00:00Z,2020-06-10T18:38:31.878Z,4.0,1650000,3323.0,2.0,1950.0,1835.0,49,...,Pacific Time in the United States,Lakeshore Beauty. Expansive 4 bedrooms & 2 b...,-122.488759,37.733202,Mark Campana,Anchor Realty,Sisley Wong,Grobecker Holland Int'l,2020,6
17470,/CA/San-Francisco/60-Rausch-St-94103/unit-401/...,2025-02-21T08:00:00Z,2025-01-09T21:59:09.430Z,3.0,1388000,16836.0,2.5,2002.0,1703.0,43,...,Pacific Time in the United States,Industrial chic meets modern living in this hi...,-122.409700,37.776144,Michael Shaw,Vanguard Properties,Joseph Hayden,Harper Real Estate,2025,1


Export Data

In [ ]:
from google.colab import files

# export to a csv
df.to_csv('housing_data-2.csv')
files.download('housing_data-2.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>